In [1]:
%%time
import math
import torch

from botorch.test_functions import SixHumpCamel
from botorch.fit import fit_gpytorch_mll
from botorch.models import SingleTaskGP
from botorch.utils.transforms import standardize, normalize
from gpytorch.mlls import ExactMarginalLogLikelihood
from botorch.acquisition import (
    ExpectedImprovement,
    ProbabilityOfImprovement,
    qMaxValueEntropy,
)
import matplotlib.pyplot as plt
from botorch.optim import optimize_acqf
from botorch.acquisition import qNoisyExpectedImprovement, qProbabilityOfImprovement
from time import time
import cProfile, io, pstats
from pstats import SortKey
import scipy

SMOKE_TEST = True

torch.manual_seed(123456)
torch.set_default_dtype(torch.double)

bounds = torch.tensor(SixHumpCamel._bounds).T
bounds_norm = torch.tensor([[0.0, 0.0], [1.0, 1.0]])
train_X = bounds[0] + (bounds[1] - bounds[0]) * torch.rand(5, 2)
train_Y = SixHumpCamel(negate=True)(train_X).unsqueeze(-1)

train_X = normalize(train_X, bounds=bounds)
train_Y = standardize(train_Y + 0.05 * torch.randn_like(train_Y))

model = SingleTaskGP(train_X, train_Y)
mll = ExactMarginalLogLikelihood(model.likelihood, model)
fit_gpytorch_mll(mll, max_attempts=10)

from botorch.acquisition.max_value_entropy_search import qLowerBoundMaxValueEntropy

candidate_set_size = 1000 if not SMOKE_TEST else 5
candidate_set = torch.rand(
    candidate_set_size, bounds_norm.size(1), device=bounds.device, dtype=bounds.dtype
)
qGIBBON = qLowerBoundMaxValueEntropy(model, candidate_set)
scipy.__version__

CPU times: user 2.09 s, sys: 2.3 s, total: 4.38 s
Wall time: 2.98 s


'1.7.3'

In [2]:
%%time

# prep different acqusition functions
candidate_set = torch.rand(
    10000, bounds.size(1), device=bounds.device, dtype=bounds.dtype
)
acqs = {}
# acqs["aa"] = qLowerBoundMaxValueEntropy(model, candidate_set)
# acqs["MES"] = qMaxValueEntropy(model, candidate_set)
acqs["EI"] = qNoisyExpectedImprovement(model, train_X)
# acqs["PI"] = qProbabilityOfImprovement(model, best_f=train_Y.max())

# prep grid to evaluate acq functions
n = 100 if not SMOKE_TEST else 2
xv, yv = torch.meshgrid([torch.linspace(0, 1, n), torch.linspace(0, 1, n)])
test_x = torch.stack([xv.reshape(n * n, 1), yv.reshape(n * n, 1)], -1)

# eval and maximise acq functions
test_x.requires_grad=True
with cProfile.Profile() as pr:
    for acq in acqs.values():
#         acq(test_x).detach().reshape(n, n)
#         optimize_acqf(
#             acq_function=acq,
#             bounds=bounds_norm,
#             q=3,
#             num_restarts=5,
#             raw_samples=100,
#             sequential=True,
#         )
        for _ in range(500):
            vals = acq(test_x.clone())
            vals.sum().backward()

sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr).sort_stats(sortby)
ps.print_stats()

scipy.__version__

/opt/anaconda3/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/native/TensorShape.cpp:3484.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


         693628 function calls (634127 primitive calls) in 0.719 seconds

   Ordered by: cumulative time

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
 2500/500    0.003    0.000    0.446    0.001 /opt/anaconda3/lib/python3.9/site-packages/torch/nn/modules/module.py:1494(_call_impl)
      500    0.001    0.000    0.445    0.001 /opt/anaconda3/lib/python3.9/site-packages/botorch/utils/transforms.py:326(decorated)
      500    0.003    0.000    0.445    0.001 /opt/anaconda3/lib/python3.9/site-packages/botorch/utils/transforms.py:266(decorated)
      500    0.003    0.000    0.434    0.001 /opt/anaconda3/lib/python3.9/site-packages/botorch/acquisition/monte_carlo.py:325(forward)
      500    0.003    0.000    0.285    0.001 /opt/anaconda3/lib/python3.9/site-packages/botorch/models/gpytorch.py:340(posterior)
      500    0.001    0.000    0.270    0.001 /opt/anaconda3/lib/python3.9/site-packages/torch/_tensor.py:428(backward)
      500    0.001    0.000    0.269 

'1.7.3'